In [1]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Image
import time
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Load the Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_image(prompt):
    start_time = time.time()
    
    with tqdm(total=100, desc="Generating image", bar_format='{l_bar}{bar}') as pbar:
        for i in range(10):
            time.sleep(0.1)  # Simulate steps in generation process
            pbar.update(10)
        image = pipe(prompt).images[0]
    
    end_time = time.time()
    
    # Save the image
    image_path = f"generated_image_{int(time.time())}.png"
    image.save(image_path)
    
    print(f"Image generated in {end_time - start_time:.2f} seconds")
    print(f"Image saved as {image_path}")
    
    # Display the image
    display(Image(filename=image_path))

# Create widgets for user interaction
text_input = widgets.Text(description="Image description:", style={'description_width': 'initial'})
generate_button = widgets.Button(description="Generate")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        generate_image(text_input.value)

generate_button.on_click(on_button_click)

# Display the UI
display(widgets.VBox([text_input, generate_button, output]))


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Image
import time
import concurrent.futures
from functools import partial
import os
from tqdm import tqdm

# Global variables
model_id = "runwayml/stable-diffusion-v1-5"
pipe = None

def initialize_pipeline():
    global pipe
    if pipe is None:
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
        pipe = pipe.to("cpu")
        pipe.enable_attention_slicing()  # Reduces memory usage
    return pipe

def generate_image_chunk(prompt, chunk_idx, num_chunks):
    pipe = initialize_pipeline()
    
    # Split the image generation process
    height = 512 // num_chunks
    top = height * chunk_idx
    
    with torch.no_grad():
        image = pipe(
            prompt,
            num_inference_steps=20,  # Reduced for faster generation
            height=512,
            width=512,
            callback_on_step_end=lambda *args: None,  # Accept any number of arguments
        ).images[0].crop((0, top, 512, top + height))
    
    return image, chunk_idx

def generate_image(prompt, num_threads=4):
    start_time = time.time()
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(partial(generate_image_chunk, prompt, i, num_threads))
            for i in range(num_threads)
        ]
        
        chunks = []
        for future in tqdm(concurrent.futures.as_completed(futures), total=num_threads, desc="Generating image chunks"):
            chunk, idx = future.result()
            chunks.append((chunk, idx))
    
    # Combine image chunks
    chunks.sort(key=lambda x: x[1])
    full_image = Image.new('RGB', (512, 512))
    for chunk, idx in chunks:
        full_image.paste(chunk, (0, 512 // num_threads * idx))
    
    end_time = time.time()
    
    # Save the image
    os.makedirs("generated_images", exist_ok=True)
    image_path = f"generated_images/generated_image_{int(time.time())}.png"
    full_image.save(image_path)
    
    print(f"Image generated in {end_time - start_time:.2f} seconds")
    print(f"Image saved as {image_path}")
    
    # Display the image
    display(full_image)

def main():
    while True:
        user_prompt = input("Enter a description for the image you want to generate (or 'quit' to exit): ")
        if user_prompt.lower() == 'quit':
            break
        generate_image(user_prompt)

if __name__ == "__main__":
    main()
    print("Thank you for using the Stable Diffusion image generator!")

Generating image chunks:   0%|          | 0/4 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]